In [1]:
from liftingcast import LiftingCast

L = LiftingCast()


In [2]:

L.fetch_meets()



In [3]:
L.fetch_lifters()


fetched 4 lifters for Connecticut Youth State Championships
fetched 0 lifters for Laconia Regional
fetched 0 lifters for Send it Saturday - Broward Barbell Center
fetched 22 lifters for Wolf Pack Powerlifting Novice Cup
fetched 26 lifters for ELITE PRO
fetched 24 lifters for XVII Pacific Games - Honiara, Solomon Islands (Day 2)
fetched 30 lifters for 2023 USA Powerlifting England - EuroTour Round 1 - The Inauguration (Day 2)
fetched 32 lifters for XVII Pacific Games - Honiara, Solomon Islands (Day 3)
fetched 37 lifters for XVII Pacific Games - Honiara, Solomon Islands (Day 1)
fetched 31 lifters for 2023 USA Powerlifting Philadelphia Love Championship (PA-2023-28)
fetched 12 lifters for Holiday Bash
fetched 46 lifters for L'IMPÉRIUM
fetched 10 lifters for 2023 Powerlifting America Prime Power Summit - 2023-TX-08
fetched 47 lifters for Drug Tested Holiday Bash
fetched 45 lifters for Barbell Barbell Winter Classic
fetched 39 lifters for Loaded Lifting USAPL Teen and Academic Iron Games 20

In [12]:
import awswrangler as wr
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
region_name = os.environ.get('AWS_REGION')
lifter_table_name = os.environ.get('AWS_DYNAMO_TABLE_NAME')
boto3.setup_default_session(region_name=region_name)



In [10]:

stored_lifters = wr.dynamodb.read_items(table_name=lifter_table_name, allow_full_scan=True)


c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\awswrangler\dynamodb\_read.py:409: UserWarning: Attempting DynamoDB Scan operation with arguments:
{'ConsistentRead': False}
  warnings.warn(
c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\awswrangler\_distributed.py:105: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [6]:

import pandas as pd

new_lifters = pd.DataFrame(L.lifters)

In [7]:
lifters_to_delete = pd.DataFrame()
lifters_to_insert = pd.DataFrame()
if 'lifter_id' in new_lifters.columns and 'lifter_id' in stored_lifters.columns:
    lifters_to_delete = stored_lifters[~stored_lifters.lifter_id.isin(new_lifters.lifter_id)]
    lifters_to_insert = new_lifters[~new_lifters.lifter_id.isin(stored_lifters.lifter_id)]


In [8]:

if 'lifter_id' in lifters_to_delete.columns:
    wr.dynamodb.delete_items(
        items=lifters_to_delete.to_dict('records'), 
        table_name=lifter_table_name
    )


In [9]:
wr.dynamodb.put_df(
    df=lifters_to_insert,
    table_name=lifter_table_name
)

c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
